In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set up paths for Kaggle and Google Drive
import os
import shutil

# Define Kaggle and destination Google Drive folder paths
kaggle_json_path = "/content/drive/MyDrive/kaggle.json"  # Ensure this is the correct path to your kaggle.json
gdrive_download_folder = "/content/drive/MyDrive/Intelligent Systems Final Project"  # Replace with your desired folder in Google Drive

# Step 3: Ensure Google Drive folder exists
os.makedirs(gdrive_download_folder, exist_ok=True)

# Step 4: Move and set permissions for kaggle.json
if os.path.exists(kaggle_json_path):
    # Create the Kaggle config folder if not already created
    os.makedirs("/root/.kaggle", exist_ok=True)

    # Move the kaggle.json file to the Kaggle config folder
    shutil.copy(kaggle_json_path, "/root/.kaggle/kaggle.json")

    # Set read/write permissions for the kaggle.json file
    os.chmod("/root/.kaggle/kaggle.json", 0o600)
    print("kaggle.json file set up successfully!")
else:
    raise FileNotFoundError("kaggle.json file not found! Please upload it to your Google Drive folder before running this cell.")

# Step 5: Import Kaggle API and authenticate
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
print("Kaggle API authenticated successfully!")

# Step 6: Download Dataset
dataset_slug = "banudeep/nthuddd2"  # Replace with the correct Kaggle dataset slug

try:
    print("Downloading dataset...")
    api.dataset_download_files(dataset_slug, path=gdrive_download_folder, unzip=True)
    print(f"Dataset downloaded and unzipped successfully in: {gdrive_download_folder}")
except Exception as e:
    print(f"An error occurred: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
kaggle.json file set up successfully!
Kaggle API authenticated successfully!
Dataset URL: https://www.kaggle.com/datasets/banudeep/nthuddd2
Dataset downloaded and unzipped successfully in: /content/drive/MyDrive/Intelligent Systems Final Project


In [ ]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
kaggle_json_path = "/content/drive/MyDrive/kaggle.json"  # Path in Google Drive
kaggle_dir = "/root/.kaggle/"                           # Kaggle directory in Colab

# Check if kaggle.json exists
if os.path.exists(kaggle_json_path):
    # Create .kaggle directory if not exists
    os.makedirs(kaggle_dir, exist_ok=True)

    # Copy kaggle.json to .kaggle directory
    shutil.copy(kaggle_json_path, kaggle_dir)

    # Set permissions for the kaggle.json file
    os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

    print("kaggle.json successfully moved to /root/.kaggle/ and permissions set!")
else:
    raise FileNotFoundError("kaggle.json file not found in /content/drive/MyDrive/!")


Mounted at /content/drive
kaggle.json successfully moved to /root/.kaggle/ and permissions set!


In [ ]:
import os

# Set the Kaggle configuration directory path explicitly
os.environ["KAGGLE_CONFIG_DIR"] = "/root/.kaggle/"

# Verify the environment variable
print(f"Kaggle config directory is set to: {os.environ['KAGGLE_CONFIG_DIR']}")


Kaggle config directory is set to: /root/.kaggle/


In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

print("Kaggle API authenticated successfully!")


Kaggle API authenticated successfully!


In [ ]:
# Step 1: Import necessary libraries
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf

# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Define directories
main_dir = "/content/drive/MyDrive/Intelligent Systems Final Project/Nthuddd2 "
train_dir = os.path.join(main_dir, "train_split")
test_dir = os.path.join(main_dir, "test_split")


Mounted at /content/drive


In [ ]:
# Step 5: Set parameters for training
batch_size = 32
img_height, img_width = 224, 224
num_classes = 2  # 'drowsy' and 'notdrowsy'

# Step 6: Data preprocessing
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Step 7: Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Step 8: Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Step 9: Model summary
model.summary()

# Step 10: Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('drowsiness_detection_model.keras', monitor='val_loss', save_best_only=True)

# Step 11: Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stopping, checkpoint]
)

# Step 12: Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# # Step 13: Save the model
model.save('/content/drive/MyDrive/Intelligent Systems Final Project/drowsiness_detection_model.keras')
print("Model saved to Google Drive!")


Found 4800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      22,151,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,247,106 (84.87 MB)

 Trainable params: 22,246,146 (84.86 MB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 1495s 10s/step - accuracy: 0.9399 - loss: 0.2061 - val_accuracy: 0.6100 - val_loss: 0.6663
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1037s 7s/step - accuracy: 0.9752 - loss: 0.0735 - val_accuracy: 0.6158 - val_loss: 1.0628
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1080s 7s/step - accuracy: 0.9794 - loss: 0.0572 - val_accuracy: 0.5017 - val_loss: 3.1732
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1047s 7s/step - accuracy: 0.9807 - loss: 0.0483 - val_accuracy: 0.5700 - val_loss: 2.4615
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1105s 7s/step - accuracy: 0.9802 - loss: 0.0576 - val_accuracy: 0.8692 - val_loss: 0.3547
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1095s 7s/step - accuracy: 0.9826 - loss: 0.0423 - val_accuracy: 0.9617 - val_loss: 0.0867
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1056s 7s/step - accuracy: 0.9848 - loss: 0.0393 - val_accuracy: 0.9825 - val_loss: 0.0424
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1100s 7s/step - accuracy: 0.9826 - loss: 0.0405 - val_accura

In [ ]:
model.save('/content/drive/MyDrive/Intelligent Systems Final Project/drowsiness_detection_model.keras')
print("Model saved to Google Drive!")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Intelligent Systems Final Project/drowsiness_detection_model.keras')

# Function to preprocess the image
def preprocess_image(img_path, img_height, img_width):
    img = image.load_img(img_path, target_size=(img_height, img_width))  # Resize image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = img_array / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict
def predict_image(img_path):
    preprocessed_img = preprocess_image(img_path, img_height, img_width)
    predictions = model.predict(preprocessed_img)
    predicted_class = np.argmax(predictions)  # Get the index of the highest probability
    confidence = np.max(predictions)  # Confidence of prediction
    classes = ['drowsy', 'notdrowsy']  # Define the class labels
    return classes[predicted_class], confidence

# Test Prediction
img_path = '/content/drive/MyDrive/Intelligent Systems Final Project/Nthuddd2 /DrowsySample.jpg'  # Provide a test image path
predicted_label, confidence = predict_image(img_path)
print(f"Predicted Class: {predicted_label}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
Predicted Class: drowsy, Confidence: 1.00
